In [ ]:
import win32clipboard
import unicodedata
import csv

In [26]:
def get_clipboard_data():
    # get clipboard data
    win32clipboard.OpenClipboard()
    data = win32clipboard.GetClipboardData(win32clipboard.CF_UNICODETEXT)
    win32clipboard.CloseClipboard()
    
    return data.splitlines()

text = get_clipboard_data()
print(text)

['1⁄2 cup butter', '1 cup sugar', '2 eggs, beaten', '3 bananas, finely crushed (for serious and extreme moist and delicious, try 4 bananas)', '1 1⁄2 cups flour', '1 teaspoon baking soda', '1⁄2 teaspoon salt', '1⁄2 teaspoon vanilla (optional)']


In [ ]:
text = ['1⁄2 cup butter', '1 cup sugar', '2 eggs, beaten', '3 bananas, finely crushed (for serious and extreme moist and delicious, try 4 bananas)', '1 1⁄2 cups flour', '1 teaspoon baking soda', '1⁄2 teaspoon salt', '1⁄2 teaspoon vanilla']
print(text)

In [ ]:
class RecipeConverter:
    def __init__(self):
        # Conversion table from volumetric to metric grams
        self.conversions = self.import_conversions('gram-conversions.csv')
                
    def import_conversions(self, filename):
        with open(filename) as csvfile:
            conversion_table = list(csv.reader(csvfile, delimiter=','))

            # Remove header
            conversion_table.pop(0)

            return conversion_table
        
    def parse_line(self, ingredient):
        # From Genius recipes
        ingredient = ingredient.replace('1⁄4', '.25')
        ingredient = ingredient.replace('1⁄3', '.33')
        ingredient = ingredient.replace('1⁄2', '.5')
        ingredient = ingredient.replace('2⁄3', '.66')
        ingredient = ingredient.replace('3⁄4', '.75')

        # Standard float expressions
        ingredient = ingredient.replace('1/8', '.125')
        ingredient = ingredient.replace('1/4', '.25')
        ingredient = ingredient.replace('1/3', '.33')
        ingredient = ingredient.replace('1/2', '.5')
        ingredient = ingredient.replace('2/3', '.66')
        ingredient = ingredient.replace('3/4', '.75')
        
        # Replace abbreviations
        ingredient = ingredient.replace('tbsp','tablespoon')
        ingredient = ingredient.replace('tsp','teaspoon')
        ingredient = ingredient.replace('oz','ounces')

        return ingredient

    def convert_ingredient(self, line, unit, conversion):
        # Extract number from measurement
        number_string = line.split(unit)[0].replace(' ','')
        number_float = float(number_string)

        # Convert butter cup to grams
        converted = number_float * float(conversion)

        # Construct the output ingredient line with original line
        line_out = '%.1f g%s'%(converted, line.split(unit)[1].strip('s'))
        return line_out

    def parse_recipe(self, text):
        output = []

        for recipe_line in text:
            flag_converted = False

            # For each recipe line item, look it up in the conversion table
            for ingredient_gram in self.conversions:
                if ingredient_gram[0] in recipe_line:
                    flag_converted = True

                    # Convert fractions to floats and remove abbreviations
                    line = self.parse_line(recipe_line)

                    if 'cup' in line:
                        output.append(self.convert_ingredient(line, 'cup', ingredient_gram[1]))
                        break
                    elif 'tablespoon' in line:
                        output.append(self.convert_ingredient(line, 'tablespoon', ingredient_gram[2]))
                        break
                    elif 'teaspoon' in line:
                        output.append(self.convert_ingredient(line, 'teaspoon', ingredient_gram[3]))
                        break
                    else:
                        flag_converted = False

            # If nothing was found, then use original line
            if not flag_converted:
                output.append(recipe_line)

        return output

rc = RecipeConverter()
rc.parse_recipe(text)

In [ ]:
def display_lines(text_list):
    for line in text_list:
        print(line)
        
    print()

In [27]:
text = get_clipboard_data()
print('***ORIGINAL***')
display_lines(text)

***ORIGINAL***
1⁄2 cup butter
1 cup sugar
2 eggs, beaten
3 bananas, finely crushed (for serious and extreme moist and delicious, try 4 bananas)
1 1⁄2 cups flour
1 teaspoon baking soda
1⁄2 teaspoon salt
1⁄2 teaspoon vanilla (optional)



In [28]:
print('***CONVERTED***')
rc = RecipeConverter()
display_lines(rc.parse_recipe(text))

***CONVERTED***
113.5 g butter
201.0 g sugar
2 eggs, beaten
3 bananas, finely crushed (for serious and extreme moist and delicious, try 4 bananas)
195.0 g flour
6.0 g baking soda
2.5 g salt
2.1 g vanilla (optional)



In [ ]:
'<br>'.join(rc.parse_recipe(text))